# 利用linear regression及手刻Gradient Descent 預測pm2.5
https://ntumlta.github.io/2017fall-ml-hw1/

In [1]:
import pandas as pd
import numpy as np
from numpy.linalg import inv
import random
import math
import sys
import csv

In [16]:
file_path = '/Users/PK/.kaggle/competitions/pm2.5/'
df_train = pd.read_csv(file_path + "train.csv",encoding='big5')
df_train.head()

,日期,測站,測項,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
0,2014/1/1,豐原,AMB_TEMP,14,14,14,13,12,12,12,...,22,22,21,19,17,16,15,15,15,15
1,2014/1/1,豐原,CH4,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,2014/1/1,豐原,CO,0.51,0.41,0.39,0.37,0.35,0.3,0.37,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,2014/1/1,豐原,NMHC,0.2,0.15,0.13,0.12,0.11,0.06,0.1,...,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08
4,2014/1/1,豐原,NO,0.9,0.6,0.5,1.7,1.8,1.5,1.9,...,2.5,2.2,2.5,2.3,2.1,1.9,1.5,1.6,1.8,1.5


In [3]:
index =df_train["測項"] == "PM2.5"
df_train[index]

,日期,測站,測項,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
9,2014/1/1,豐原,PM2.5,26,39,36,35,31,28,25,...,36,45,42,49,45,44,41,30,24,13
27,2014/1/2,豐原,PM2.5,21,23,30,30,22,18,13,...,53,43,43,45,46,32,16,19,22,26
45,2014/1/3,豐原,PM2.5,19,25,27,20,16,14,15,...,32,36,34,45,40,41,23,29,23,37
63,2014/1/4,豐原,PM2.5,27,27,14,20,22,24,26,...,62,55,56,67,78,83,90,75,85,82
81,2014/1/5,豐原,PM2.5,80,80,76,81,75,66,70,...,64,73,57,57,53,70,70,60,68,66
99,2014/1/6,豐原,PM2.5,66,55,46,46,46,42,35,...,62,59,61,50,60,62,62,50,45,43
117,2014/1/7,豐原,PM2.5,41,32,35,31,33,25,36,...,62,61,47,54,57,59,61,43,39,38
135,2014/1/8,豐原,PM2.5,48,53,60,60,52,47,45,...,35,36,36,32,33,33,34,21,15,13
153,2014/1/9,豐原,PM2.5,18,15,9,9,14,15,20,...,20,20,20,18,18,22,15,11,1,1
171,2014/1/10,豐原,PM2.5,6,7,7,0,0,1,1,...,23,20,18,15,16,10,6,11,22,28


In [4]:
#18種污染物質
#20天資料
index =df_train["日期"] == "2014/1/1"
df_train[index]["測項"]

0       AMB_TEMP
1            CH4
2             CO
3           NMHC
4             NO
5            NO2
6            NOx
7             O3
8           PM10
9          PM2.5
10      RAINFALL
11            RH
12           SO2
13           THC
14         WD_HR
15    WIND_DIREC
16    WIND_SPEED
17         WS_HR
Name: 測項, dtype: object

In [5]:
df_train = df_train.replace({"NR":0}) # 雨量 NR 設為0

df_list = []
for d in range(20):
    d = d+1
    index =df_train["日期"] == "2014/1/"+str(d)
    df = df_train[index].iloc[:,3:].reset_index().drop(["index"], axis=1)
    df_list.append(df)
# df_list[19]
df_result =pd.concat(df_list, axis=1,ignore_index=True)
df_result = df_result.T
df_result.columns =df_train[index]["測項"]
df_result.head(10)

測項,AMB_TEMP,CH4,CO,NMHC,NO,NO2,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,THC,WD_HR,WIND_DIREC,WIND_SPEED,WS_HR
0,14,1.8,0.51,0.2,0.9,16,17,16,56,26,0,77,1.8,2,37,35,1.4,0.5
1,14,1.8,0.41,0.15,0.6,9.2,9.8,30,50,39,0,68,2,2,80,79,1.8,0.9
2,14,1.8,0.39,0.13,0.5,8.2,8.7,27,48,36,0,67,1.7,2,57,2.4,1,0.6
3,13,1.8,0.37,0.12,1.7,6.9,8.6,23,35,35,0,74,1.6,1.9,76,55,0.6,0.3
4,12,1.8,0.35,0.11,1.8,6.8,8.5,24,25,31,0,72,1.9,1.9,110,94,1.7,0.6
5,12,1.8,0.3,0.06,1.5,3.8,5.3,28,12,28,0,73,1.4,1.8,106,116,2.5,1.9
6,12,1.8,0.37,0.1,1.9,6.9,8.8,24,4,25,0,74,1.5,1.9,101,106,2.5,2
7,12,1.8,0.47,0.13,2.2,7.8,9.9,22,2,20,0,73,1.6,1.9,104,94,2,2
8,15,1.8,0.78,0.26,6.6,15,22,21,11,19,0,66,5.1,2.1,124,232,0.6,0.5
9,17,1.8,0.74,0.23,7.9,21,29,29,38,30,0,56,15,2,46,153,0.8,0.3


In [6]:
df_result.describe()
# df_test.describe()

測項,AMB_TEMP,CH4,CO,NMHC,NO,NO2,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,THC,WD_HR,WIND_DIREC,WIND_SPEED,WS_HR
count,480,480,480,480,480,480,480,480,480,480,480,480,480,480,480,480,480,480
unique,20,3,74,46,70,106,107,77,129,88,5,55,90,9,167,181,41,41
top,14,1.8,0.3,0.13,1.1,11,11,30,25,23,0,73,1.2,1.9,58,63,1.6,1
freq,70,391,20,38,34,30,43,27,14,17,476,29,19,230,15,12,29,31


In [7]:
def xy(df):
    x=[]
    for i in range(len(df)-9):
        x.append(df[i+1:i+10].astype("float64").values.flatten())
    print(len(x),len(x[0]))

    y= df_result["PM2.5"][9:].values
    len(y)
    return x,y
x,y = xy(df_result)
x[0]

471 162


array([1.40e+01, 1.80e+00, 4.10e-01, 1.50e-01, 6.00e-01, 9.20e+00,
       9.80e+00, 3.00e+01, 5.00e+01, 3.90e+01, 0.00e+00, 6.80e+01,
       2.00e+00, 2.00e+00, 8.00e+01, 7.90e+01, 1.80e+00, 9.00e-01,
       1.40e+01, 1.80e+00, 3.90e-01, 1.30e-01, 5.00e-01, 8.20e+00,
       8.70e+00, 2.70e+01, 4.80e+01, 3.60e+01, 0.00e+00, 6.70e+01,
       1.70e+00, 2.00e+00, 5.70e+01, 2.40e+00, 1.00e+00, 6.00e-01,
       1.30e+01, 1.80e+00, 3.70e-01, 1.20e-01, 1.70e+00, 6.90e+00,
       8.60e+00, 2.30e+01, 3.50e+01, 3.50e+01, 0.00e+00, 7.40e+01,
       1.60e+00, 1.90e+00, 7.60e+01, 5.50e+01, 6.00e-01, 3.00e-01,
       1.20e+01, 1.80e+00, 3.50e-01, 1.10e-01, 1.80e+00, 6.80e+00,
       8.50e+00, 2.40e+01, 2.50e+01, 3.10e+01, 0.00e+00, 7.20e+01,
       1.90e+00, 1.90e+00, 1.10e+02, 9.40e+01, 1.70e+00, 6.00e-01,
       1.20e+01, 1.80e+00, 3.00e-01, 6.00e-02, 1.50e+00, 3.80e+00,
       5.30e+00, 2.80e+01, 1.20e+01, 2.80e+01, 0.00e+00, 7.30e+01,
       1.40e+00, 1.80e+00, 1.06e+02, 1.16e+02, 2.50e+00, 1.90e

In [8]:
x = np.array(x,dtype=np.float64)
y = np.array(y,dtype=np.float64)

In [9]:
# add bias
x = np.concatenate((np.ones((x.shape[0], 1)), x), axis=1)

In [10]:
# Initial weight & other hyperparams
w = np.zeros(len(x[0]))
lr = 10
# iteration =100
iteration = 10000

In [11]:
#check your ans with close form solution
# use close form to check whether ur gradient descent is good
# however, this cannot be used in hw1.sh
# w = np.matmul(np.matmul(inv(np.matmul(x.T,x)),x.T),y)

In [12]:
# start training
# Gradient Descent 
s_gra = np.zeros(len(x[0]))

for i in range(iteration):
    y_h = np.dot(x, w)
    loss = y_h - y
    cost = np.sum(loss**2) / len(x)
    cost_a = math.sqrt(cost)
    gra = np.dot(x.T, loss)
    s_gra += gra**2
    ada = np.sqrt(s_gra)
    w = w - lr * gra / ada
#     print('iteration: %d | Cost: %f  ' % (i, cost_a))
print(np.mean(loss))

0.00100898590249853


In [358]:
# save model
np.save('model.npy', w)
# read model
w = np.load('model.npy')

In [17]:
df_test = pd.read_csv(file_path + "test.csv", header=None)
df_test = df_test.replace({"NR":0}) # 雨量 NR 設為0
df_test.head(18)

,0,1,2,3,4,5,6,7,8,9,10
0,id_0,AMB_TEMP,15,14,14,13,13,13,13,13,12
1,id_0,CH4,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,id_0,CO,0.36,0.35,0.34,0.33,0.33,0.34,0.34,0.37,0.42
3,id_0,NMHC,0.11,0.09,0.09,0.1,0.1,0.1,0.1,0.11,0.12
4,id_0,NO,0.6,0.4,0.3,0.3,0.3,0.7,0.8,0.8,0.9
5,id_0,NO2,9.3,7.1,6.1,5.7,5.5,5.3,5.5,7.1,7.5
6,id_0,NOx,9.9,7.5,6.4,5.9,5.8,6,6.2,7.8,8.4
7,id_0,O3,36,44,45,44,44,44,43,40,38
8,id_0,PM10,51,51,31,40,34,51,42,36,30
9,id_0,PM2.5,27,13,24,29,41,30,29,27,28


In [18]:
df_test.iloc[0:18,2:].T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
2,15,1.8,0.36,0.11,0.6,9.3,9.9,36,51,27,0,75,1.2,1.9,116,115,2.6,2.1
3,14,1.8,0.35,0.09,0.4,7.1,7.5,44,51,13,0,71,1.2,1.8,114,113,2.2,2.4
4,14,1.8,0.34,0.09,0.3,6.1,6.4,45,31,24,0,71,1.2,1.8,112,105,2,2.2
5,13,1.8,0.33,0.1,0.3,5.7,5.9,44,40,29,0,73,1.6,1.9,109,102,1.9,1.9
6,13,1.8,0.33,0.1,0.3,5.5,5.8,44,34,41,0,74,1.5,1.9,111,106,2.4,2.3
7,13,1.8,0.34,0.1,0.7,5.3,6,44,51,30,0,74,1.5,1.9,104,106,2.4,2.3
8,13,1.8,0.34,0.1,0.8,5.5,6.2,43,42,29,0,74,1.5,1.9,107,112,2.5,2.5
9,13,1.8,0.37,0.11,0.8,7.1,7.8,40,36,27,0,74,1.6,1.9,108,113,2.8,2.5
10,12,1.8,0.42,0.12,0.9,7.5,8.4,38,30,28,0,74,1.6,1.9,104,106,2,2.3


In [19]:
x_test=[]
x_size = int(len(df_test)/18)
print(x_size)
for i in range(x_size):
    x_test.append(df_test.iloc[i*18:(i+1)*18,2:].T.values.astype("float64").flatten())
x_test = np.array(x_test)
# len(x_test)
x_test

240


array([[ 15.  ,   1.8 ,   0.36, ..., 106.  ,   2.  ,   2.3 ],
       [ 12.  ,   1.8 ,   0.46, ..., 105.  ,   3.3 ,   1.5 ],
       [  8.8 ,   1.7 ,   0.46, ...,   1.  ,   1.5 ,   1.4 ],
       ...,
       [ 12.  ,   1.8 ,   0.45, ...,  82.  ,   0.6 ,   0.6 ],
       [ 14.  ,   1.8 ,   0.57, ...,  98.  ,   1.8 ,   1.8 ],
       [ 11.  ,   1.7 ,   0.32, ..., 268.  ,   1.2 ,   0.5 ]])

In [20]:
# add bias
x_test = np.concatenate((np.ones((x_test.shape[0], 1)), x_test), axis=1)
print(len(x[0]),len(x_test[0]))

163 163


In [21]:
# get ans.csv with your model
ans = []
for i in range(len(x_test)):
    ans.append(["id_" + str(i)])
    a = np.dot(w, x_test[i])
    ans[i].append(a)

In [23]:
filename = "predict2.csv"
with open(filename, "w+") as text:
    s = csv.writer(text, delimiter=',', lineterminator='\n')
    s.writerow(["id", "value"])
    for i in range(len(ans)):
        s.writerow(ans[i])

In [24]:
ans[0:10]

[['id_0', 28.16417222724241],
 ['id_1', 62.99572220964621],
 ['id_2', 20.264968186565273],
 ['id_3', 29.463077923686914],
 ['id_4', 12.036730764677694],
 ['id_5', 29.465949354380854],
 ['id_6', 40.798528912539005],
 ['id_7', 17.782498818813707],
 ['id_8', 51.5280068954892],
 ['id_9', 33.67358783605102]]